Выберите веб-сайт с табличными данными, который вас интересует.
Напишите код Python, использующий библиотеку requests для отправки HTTP GET-запроса на сайт и получения HTML-содержимого страницы.
Выполните парсинг содержимого HTML с помощью библиотеки lxml, чтобы извлечь данные из таблицы.
Сохраните извлеченные данные в CSV-файл с помощью модуля csv.

In [1]:
import requests
import json
import time
import csv
import codecs
from pymongo import MongoClient
from lxml import html

In [2]:
url = 'https://www.soccer.ru/stats/strikers/208298/1353714?page='
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'


response = requests.get(url=url, headers = {'User-Agent' : user_agent})
response.status_code

print(response.status_code)


tree = html.fromstring(response.content)
table_rows = tree.xpath('//table[@class="site-table turnir-stat-table"]/tbody/tr')


data = []

for row in table_rows:
    columns =  row.xpath('.//td/text()')
    stat = {}
    try:
        stat['Rank']= columns[0].strip('.')
    except:
        stat['Rank']= ''
    try:
        stat['Player']= row.xpath('.//td[2]/a/text()')[0].strip()
    except:
        stat['Player']= ''
    try:
        stat['Position']= columns[3].strip()
    except:
        stat['Position']= ''        
    try:
        stat['Team']= row.xpath('.//td[4]/a/text()')[0].strip()
    except:
        stat['Team']= ''
    try:
        stat['Games']= columns[6].strip()
    except:
        stat['Games']= ''
    try:
        stat['Goals']= columns[7].strip()
    except:
        stat['Goals']= ''
    try:
        stat['Penalties']= columns[8].strip()
    except:
        stat['Penalties']= ''
    try:
        stat['Goals/Games']= columns[9].strip()
    except:
        stat['Goals/Games']= ''
    
    data.append(stat)
    
print(data)

200
[{'Rank': '1', 'Player': 'Холанд Эрлинг', 'Position': 'Нападающий', 'Team': 'Манчестер Сити', 'Games': '15', 'Goals': '14', 'Penalties': '3', 'Goals/Games': '0.93'}, {'Rank': '2', 'Player': 'Салах Мохамед', 'Position': 'Нападающий', 'Team': 'Ливерпуль', 'Games': '20', 'Goals': '14', 'Penalties': '4', 'Goals/Games': '0.70'}, {'Rank': '3', 'Player': 'Соланке Доминик', 'Position': 'Нападающий', 'Team': 'Борнмут', 'Games': '20', 'Goals': '12', 'Penalties': '1', 'Goals/Games': '0.60'}, {'Rank': '4', 'Player': 'Сон Хын-Мин', 'Position': 'Нападающий', 'Team': 'Тоттенхэм', 'Games': '20', 'Goals': '12', 'Penalties': '1', 'Goals/Games': '0.60'}, {'Rank': '5', 'Player': 'Боуэн Джаррод', 'Position': 'Нападающий', 'Team': 'Вест Хэм', 'Games': '20', 'Goals': '11', 'Penalties': '0', 'Goals/Games': '0.55'}, {'Rank': '6', 'Player': 'Исак Александер', 'Position': 'Нападающий', 'Team': 'Ньюкасл', 'Games': '16', 'Goals': '10', 'Penalties': '2', 'Goals/Games': '0.63'}, {'Rank': '7', 'Player': 'Хван Хи-

In [3]:
def scrape_page_data(url):
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    response = requests.get(url=url, headers = {'User-Agent' : user_agent})
    tree = html.fromstring(response.content)
    table_rows = tree.xpath('//table[@class="site-table turnir-stat-table"]/tbody/tr')
    data = []
    

    for row in table_rows:
        columns =  row.xpath('.//td/text()')
        stat = {}
        try:
            stat['Rank']= columns[0].strip('.')
        except:
            stat['Rank']= ''
        try:
            stat['Player']= row.xpath('.//td[2]/a/text()')[0].strip()
        except:
            stat['Player']= ''
        try:
            stat['Position']= columns[3].strip()
        except:
            stat['Position']= ''        
        try:
            stat['Team']= row.xpath('.//td[4]/a/text()')[0].strip()
        except:
            stat['Team']= ''
        try:
            stat['Games']= columns[6].strip()
        except:
            stat['Games']= ''
        try:
            stat['Goals']= columns[7].strip()
        except:
            stat['Goals']= ''
        try:
            stat['Penalties']= columns[8].strip()
        except:
            stat['Penalties']= ''
        try:
            stat['Goals/Games']= columns[9].strip()
        except:
            stat['Goals/Games']= ''

        data.append(stat)
        
    return data

In [4]:
def save_data_to_mongodb(data):
    client = MongoClient('localhost', 27017)
    db = client['hw_4']
    collection = db['goals_statistics_2024_england']
    collection.insert_many(data)

In [5]:
base_url = 'https://www.soccer.ru/stats/strikers/208298/1353714?page='
agent_ = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'

    
for page in range(0,4):
    print(f'Scraping page - {page}...')
    url = base_url + str(page)
    data = scrape_page_data(url) #exctract data
    print(data)
    with open('hw_4_result.csv', 'a', newline='') as f:
        fieldnames = ['Rank', 'Player', 'Position', 'Team', 'Games', 'Goals', 'Penalties', 'Goals/Games']
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for row in data:
            writer.writerow(row)
    save_data_to_mongodb(data) #loat to mongo DB
    time.sleep(5) # like human

print('Well done')


Scraping page - 0...
[{'Rank': '1', 'Player': 'Холанд Эрлинг', 'Position': 'Нападающий', 'Team': 'Манчестер Сити', 'Games': '15', 'Goals': '14', 'Penalties': '3', 'Goals/Games': '0.93'}, {'Rank': '2', 'Player': 'Салах Мохамед', 'Position': 'Нападающий', 'Team': 'Ливерпуль', 'Games': '20', 'Goals': '14', 'Penalties': '4', 'Goals/Games': '0.70'}, {'Rank': '3', 'Player': 'Соланке Доминик', 'Position': 'Нападающий', 'Team': 'Борнмут', 'Games': '20', 'Goals': '12', 'Penalties': '1', 'Goals/Games': '0.60'}, {'Rank': '4', 'Player': 'Сон Хын-Мин', 'Position': 'Нападающий', 'Team': 'Тоттенхэм', 'Games': '20', 'Goals': '12', 'Penalties': '1', 'Goals/Games': '0.60'}, {'Rank': '5', 'Player': 'Боуэн Джаррод', 'Position': 'Нападающий', 'Team': 'Вест Хэм', 'Games': '20', 'Goals': '11', 'Penalties': '0', 'Goals/Games': '0.55'}, {'Rank': '6', 'Player': 'Исак Александер', 'Position': 'Нападающий', 'Team': 'Ньюкасл', 'Games': '16', 'Goals': '10', 'Penalties': '2', 'Goals/Games': '0.63'}, {'Rank': '7', 'P

Scraping page - 2...
[{'Rank': '101', 'Player': 'Харрисон Джек', 'Position': 'Полузащитник', 'Team': 'Эвертон', 'Games': '15', 'Goals': '2', 'Penalties': '0', 'Goals/Games': '0.13'}, {'Rank': '102', 'Player': 'Аке Натан', 'Position': 'Защитник', 'Team': 'Манчестер Сити', 'Games': '16', 'Goals': '2', 'Penalties': '0', 'Goals/Games': '0.13'}, {'Rank': '103', 'Player': 'Берн Дэн', 'Position': 'Защитник', 'Team': 'Ньюкасл', 'Games': '16', 'Goals': '2', 'Penalties': '0', 'Goals/Games': '0.13'}, {'Rank': '104', 'Player': 'Жоэлинтон', 'Position': 'Нападающий', 'Team': 'Ньюкасл', 'Games': '16', 'Goals': '2', 'Penalties': '0', 'Goals/Games': '0.13'}, {'Rank': '105', 'Player': 'Клюйверт Джастин', 'Position': 'Нападающий', 'Team': 'Борнмут', 'Games': '16', 'Goals': '2', 'Penalties': '0', 'Goals/Games': '0.13'}, {'Rank': '106', 'Player': 'Миколенко Виталий', 'Position': 'Защитник', 'Team': 'Эвертон', 'Games': '16', 'Goals': '2', 'Penalties': '0', 'Goals/Games': '0.13'}, {'Rank': '107', 'Player': '

Well done
